In [221]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)

plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings
do_compile = True
threads = 7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Solve the model

In [228]:
# compile c++ files
T = 5
specs = {  
    'model 1, limited':{'latexname':'limited', 'par':{'kappa1': 0.5, 'kappa2': 0.0, 'T':T,'threads':threads,'bargaining':1}},
    'model 2, limited':{'latexname':'limited', 'par':{'wage_const_w': 1.7, 'wage_const_m': 1.7, 'wage_K_w': 0.95, 'wage_K_m': 1.4, 'T':T,'threads':threads,'bargaining':1}},
    'model 3, limited':{'latexname':'limited', 'par':{'wage_const_w': 1.7, 'wage_const_m': 1.7, 'wage_K_w': 0.95, 'wage_K_m': 1.4, 'kappa1': 0.5, 'kappa2': 0.0, 'T':T,'threads':threads,'bargaining':1}},
}





# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models[name] = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models[name].spec = spec

    compile_now = True if do_compile & (m==0) else False
    models[name].link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models[name].solve() 

model 1, limited loading... solving...
CPU times: total: 34min 9s
Wall time: 8min 44s
model 2, limited loading... solving...
CPU times: total: 33min 50s
Wall time: 7min 29s
model 3, limited loading... solving...
CPU times: total: 34min 14s
Wall time: 6min 53s


In [236]:

model = models['model 1, limited']
model.sim.init_A[:] =5.0 
print(model.par.kappa1)
model.simulate()
print('model1')
print(f'Couple {np.mean(model.sim.couple,0)}')
print(f'Laborw {np.mean(model.sim.labor_w,0)}')
print(f'Laborm {np.mean(model.sim.labor_m,0)}')
print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
print(f'HKw {np.mean(model.sim.Kw,0)}')
print(f'HKm {np.mean(model.sim.Km,0)}')

model = models['model 2, limited']
model.simulate()
print('model2')
print(f'Couple {np.mean(model.sim.couple,0)}')
print(f'Laborw {np.mean(model.sim.labor_w,0)}')
print(f'Laborm {np.mean(model.sim.labor_m,0)}')
print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
print(f'HKw {np.mean(model.sim.Kw,0)}')
print(f'HKm {np.mean(model.sim.Km,0)}')


model = models['model 3, limited']
model.simulate()
print('model3')
print(f'Couple {np.mean(model.sim.couple,0)}')
print(f'Laborw {np.mean(model.sim.labor_w,0)}')
print(f'Laborm {np.mean(model.sim.labor_m,0)}')
print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
print(f'HKw {np.mean(model.sim.Kw,0)}')
print(f'HKm {np.mean(model.sim.Km,0)}')



0.5
model1
Couple [1.00e+00 1.44e-03 8.60e-04 6.00e-04 5.20e-04]
Laborw [0.18217749 0.44151532 0.45127965 0.44760632 0.43019938]
Laborm [0.20519611 0.45422147 0.46493352 0.464973   0.44100646]
Consumption W [1.51512449 1.55755462 1.57760991 1.58003768 1.56588497]
Consumption M [1.51512449 1.60821111 1.62626123 1.63112896 1.61014428]
HKw [0.         0.18222415 0.60561246 0.99677239 1.34456194]
HKm [0.         0.20509613 0.63933419 1.04030085 1.40086113]
model2
Couple [0. 0. 0. 0. 0.]
Laborw [0.77093657 0.74337164 0.71290752 0.68045393 0.6450805 ]
Laborm [0.80052921 0.76607807 0.72809773 0.68872253 0.64480411]
Consumption W [1.67004441 1.71945888 1.74522183 1.74852352 1.7272844 ]
Consumption M [1.73068479 1.79854576 1.83496968 1.84191151 1.81454754]
HKw [0.         0.77113401 1.43764236 2.00761632 2.48703989]
HKm [0.         0.80013914 1.48743999 2.06670791 2.54807763]
model3
Couple [0. 0. 0. 0. 0.]
Laborw [0.94454009 0.87647762 0.80097951 0.72809728 0.64480703]
Laborm [0.97182319 0.8977

# Estimate the model

In [209]:
#Estimate the model one time
#model = models['model 3']

model.sim.init_A[:] = 6.0 # temp just to check that it works, if they have too little initial saving they will just consume everything, and saving is therefore close to zero, and the model cannot be estimated

np.random.seed(model.par.seed)
data = Estimate.create_data(model,start_p = 1, end_p = 4, to_xl = True)
data_reg = Estimate.aux_est(data,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 

Residuals from hours equation
                                 OLS Regression Results                                
Dep. Variable:        delta_log_Labor   R-squared (uncentered):                   0.452
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                          2.697e+04
Date:                Mon, 22 Jan 2024   Prob (F-statistic):                        0.00
Time:                        14:54:42   Log-Likelihood:                      2.0564e+05
No. Observations:               98028   AIC:                                 -4.113e+05
Df Residuals:                   98025   BIC:                                 -4.112e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


                                 OLS Regression Results                                
Dep. Variable:                   uhat   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                          1.289e+06
Date:                Mon, 22 Jan 2024   Prob (F-statistic):                        0.00
Time:                        14:54:45   Log-Likelihood:                      3.0638e+05
No. Observations:               65352   AIC:                                 -6.127e+05
Df Residuals:                   65341   BIC:                                 -6.126e+05
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------